#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import when, count, isnull, expr, current_date, col, date_format, year, month, lag, sum, round, max, min, datediff, explode
from pyspark.sql.window import Window

In [0]:
local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"

# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)

Wrote 17300 bytes.
Wrote 1505911 bytes.
Wrote 4018007 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'codice_nuts_1', 
                                               'descrizione_nuts_1', 'sigla_regione', 
                                               'latitudine_regione', 'longitudine_regione',
                                               'totale_genere_maschile', 'totale_genere_femminile')

popolazione_regioni.show(2)

+-------------+---------------------+---------+---------------+
codice_nuts_2|denominazione_regione|range_eta|totale_generale|
+-------------+---------------------+---------+---------------+
 ITF1| Abruzzo| 0-15| 169897|
 ITF1| Abruzzo| 16-19| 46751|
+-------------+---------------------+---------+---------------+
only showing top 2 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[243]: [('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_2') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+---------------------+
codice_nuts_2|denominazione_regione|
+-------------+---------------------+
 ITC1| Piemonte|
 ITC2| Valle d'Aosta|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1| Bolzano|
 ITH2| Trento|
 ITH3| Veneto|
 ITH4| Friuli Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+-------------+---------------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show()

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe('totale_generale') \
                   .show()

+-------+-----------------+
summary| totale_generale|
+-------+-----------------+
 count| 210|
 mean|284007.0857142857|
 stddev|303588.4857882117|
 min| 1764|
 max| 1592109|
+-------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+---------------------+---------+---------------+
codice_nuts_2|denominazione_regione|range_eta|totale_generale|
+-------------+---------------------+---------+---------------+
 0| 0| 0| 0|
+-------------+---------------------+---------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_nuts_1', 'codice_regione', 
                                             'lat', 'long', 'note_test', 'note_casi',
                                             'note', 'casi_da_sospetto_diagnostico', 
                                             'casi_da_screening','casi_testati', 
                                             'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 
                                             'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 
                                             'tamponi_test_antigenico_rapido', 
                                             'isolamento_domiciliare', 
                                             'variazione_totale_positivi', 
                                             'dimessi_guariti', 'totale_casi')
dati_covid_regioni.show(2)

+-------------------+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|totale_positivi|nuovi_positivi|deceduti|tamponi|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 5| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
only showing top 2 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[249]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('deceduti', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_2', 'string')]

In [0]:
dati_covid_regioni = dati_covid_regioni.withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd').cast('date'))

In [0]:
dati_covid_regioni.dtypes

Out[251]: [('data', 'date'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('deceduti', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()

print('Il numero di regioni è:', num_regioni)
print()

dati_covid_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_2') \
                    .show(5)

Il numero di regioni è: 21

+-------------+---------------------+
codice_nuts_2|denominazione_regione|
+-------------+---------------------+
 null| Abruzzo|
 null| Lombardia|
 null| Piemonte|
 null| Emilia-Romagna|
 null| Calabria|
+-------------+---------------------+
only showing top 5 rows

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_2')

nuts_2 = popolazione_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                            .count() \
                            .drop('count')

dati_covid_regioni = dati_covid_regioni.join(nuts_2, on='denominazione_regione', how='left')

dati_covid_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                  .count() \
                  .drop('count') \
                  .orderBy('codice_nuts_2').show(num_regioni)

+-------------+---------------------+
codice_nuts_2|denominazione_regione|
+-------------+---------------------+
 ITC1| Piemonte|
 ITC2| Valle d'Aosta|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1| Bolzano|
 ITH2| Trento|
 ITH3| Veneto|
 ITH4| Friuli Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+-------------+---------------------+

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_covid_regioni.describe(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'totale_positivi', 'nuovi_positivi', 'deceduti', 'tamponi']) \
                  .show()

+-------+----------------------+-----------------+--------------------+------------------+-----------------+------------------+------------------+
summary|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati| totale_positivi| nuovi_positivi| deceduti| tamponi|
+-------+----------------------+-----------------+--------------------+------------------+-----------------+------------------+------------------+
 count| 9072| 9072| 9072| 9072| 9072| 9072| 9072|
 mean| 712.4712301587301|81.71847442680776| 794.1897045855379|12468.846560846561|442.7234347442681|2470.1935626102295| 848671.4441137566|
 stddev| 1289.9373804295726|143.0112526568166| 1428.1841519339673| 22237.61213787529| 853.939588083822| 4695.206250550821|1338368.1129099485|
 min| 0| 0| 0| 0| -229| 0| 0|
 max| 12077| 1381| 13328| 164406| 11489| 32870| 9475296|
+-------+----------------------+-----------------+--------------------+------------------+-----------------+------------------+------------------+

In [0]:
dati_covid_regioni.filter(condition='nuovi_positivi < 0').show()

+---------------------+----------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|totale_positivi|nuovi_positivi|deceduti|tamponi|codice_nuts_2|
+---------------------+----------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
 Piemonte|2020-02-27| 2| 0| 2| 2| -1| 0| 156| ITC1|
 Liguria|2020-03-01| 12| 1| 13| 21| -17| 0| 121| ITC3|
 Liguria|2020-03-02| 12| 1| 13| 18| -3| 0| 121| ITC3|
 Sicilia|2020-03-02| 2| 0| 2| 5| -2| 0| 307| ITG1|
 Piemonte|2020-03-09| 222| 50| 272| 337| -10| 13| 1681| ITC1|
 Calabria|2020-04-17| 154| 7| 161| 819| -18| 73| 21657| ITF6|
 Sardegna|2020-05-04| 91| 9| 100| 653| -2| 119| 28052| ITG2|
 Basilicata|2020-05-07| 48| 2| 50| 155| -16| 26| 16272| ITF5|
 Basilicata|2020-05-08| 48| 2| 50| 152| -1| 26| 16777| ITF5|
 Marche|2020-05-19| 144| 17| 161| 2128| -3| 986| 89985| ITI3|
 Sardegna|2020-05-25| 49| 3| 52| 231| -2| 129| 51073| ITG2|
 Sardegna|2020-06-09| 9| 1| 10| 54| -1| 131| 64272| ITG2|
 Campania|2020-06-12| 61| 2| 63| 346| -229| 430| 230551| ITF3|
+---------------------+----------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|totale_positivi|nuovi_positivi|deceduti|tamponi|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+

Si verifica la presenza di *missing values* nella serie temporale.

In [0]:
min_data = dati_covid_regioni.agg(min('data').alias('minimo'))
max_data = dati_covid_regioni.agg(max('data').alias('massimo'))

date = min_data.join(max_data)

date.show()

+----------+----------+
 minimo| massimo|
+----------+----------+
2020-02-24|2021-04-30|
+----------+----------+

In [0]:
time_series = date.withColumn('data', explode(expr('sequence(minimo, massimo, interval 1 day)'))) \
                  .select('data')

time_series.show()

+----------+
 data|
+----------+
2020-02-24|
2020-02-25|
2020-02-26|
2020-02-27|
2020-02-28|
2020-02-29|
2020-03-01|
2020-03-02|
2020-03-03|
2020-03-04|
2020-03-05|
2020-03-06|
2020-03-07|
2020-03-08|
2020-03-09|
2020-03-10|
2020-03-11|
2020-03-12|
2020-03-13|
2020-03-14|
+----------+
only showing top 20 rows

In [0]:
date_covid = dati_covid_regioni.select('data') \
                               .distinct() \
                               .orderBy('data')

date_covid.join(time_series, on='data', how='anti').show()

+----+
data|
+----+
+----+

### Dati Somministrazioni Vaccini

Si elmininano le colonne che non sono di interesse

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.drop('area', 'codice_regione_ISTAT', 'sesso_maschile', 'sesso_femminile' 'codice_NUTS1')
dati_vaccini_somministrazioni.show()

+---------------------+---------------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
data_somministrazione| fornitore|fascia_anagrafica|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2| nome_area|
+---------------------+---------------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
 2020-12-27|Pfizer/BioNTech| 20-29| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 30-39| 3| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 4| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 40-49| 6| 7| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 50-59| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 60-69| 4| 13| 1| 0| 0| 0| 0| 0| 0| 0| 0| 14| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 70-79| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 80-89| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 20-29| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 30-39| 18| 27| 0| 0| 0| 0| 0| 0| 0| 0| 1| 28| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 40-49| 23| 29| 0| 0| 0| 0| 0| 0| 0| 0| 1| 30| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 50-59| 15| 26| 0| 0| 0| 0| 0| 0| 0| 0| 0| 26| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 60-69| 4| 13| 0| 0| 0| 0| 0| 0| 0| 0| 0| 13| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 20-29| 34| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 30-39| 20| 43| 0| 0| 0| 0| 0| 0| 0| 0| 0| 43| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 40-49| 27| 50| 0| 0| 0| 0| 0| 0| 0| 0| 0| 50| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 50-59| 28| 76| 1| 0| 0| 0| 0| 0| 0| 0| 0| 77| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 60-69| 16| 60| 0| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 20-29| 33| 59| 1| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 30-39| 64| 122| 2| 0| 0| 0| 0| 0| 0| 0| 1| 125| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 40-49| 87| 164| 3| 0| 0| 0| 0| 0| 0| 0| 0| 167| 0| ITF| ITF3| Campania|
+---------------------+---------------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[262]: [('data_somministrazione', 'string'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS1', 'string'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.withColumn('data_somministrazione', date_format(dati_vaccini_somministrazioni.data_somministrazione, 'yyyy-MM-dd').cast('date'))

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[264]: [('data_somministrazione', 'date'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS1', 'string'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                           .count() \
                                           .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_vaccini_somministrazioni.groupBy('codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS2') \
                             .show(num_regioni)

Il numero di regioni è: 21

+------------+--------------------+
codice_NUTS2| nome_area|
+------------+--------------------+
 ITC1| Piemonte|
 ITC2|Valle d'Aosta / V...|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1|Provincia Autonom...|
 ITH2|Provincia Autonom...|
 ITH3| Veneto|
 ITH4|Friuli-Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+------------+--------------------+

In [0]:
join1 = popolazione_regioni.join(dati_vaccini_somministrazioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') 

join2 = dati_vaccini_somministrazioni.join(popolazione_regioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_NUTS2', 'nome_area') \
                    .count() \
                    .drop('count') 

join1.join(join2, popolazione_regioni.codice_nuts_2 == dati_vaccini_somministrazioni.codice_NUTS2).show()

dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Trento', value='Trento', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Bolzano / Bozen', value='Bolzano', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Friuli-Venezia Giulia', value='Friuli Venezia Giulia', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Valle d\'Aosta / Vallée d\'Aoste', value='Valle d\'Aosta', subset='nome_area')

+-------------+---------------------+------------+--------------------+
codice_nuts_2|denominazione_regione|codice_NUTS2| nome_area|
+-------------+---------------------+------------+--------------------+
 ITC2| Valle d'Aosta| ITC2|Valle d'Aosta / V...|
 ITH2| Trento| ITH2|Provincia Autonom...|
 ITH4| Friuli Venezia Gi...| ITH4|Friuli-Venezia Gi...|
 ITH1| Bolzano| ITH1|Provincia Autonom...|
+-------------+---------------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS2') \
                             .show(num_regioni)

+------------+--------------------+
codice_NUTS2| nome_area|
+------------+--------------------+
 ITC1| Piemonte|
 ITC2| Valle d'Aosta|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1| Bolzano|
 ITH2| Trento|
 ITH3| Veneto|
 ITH4|Friuli Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                    .count() \
                    .drop('count') \
                    .orderBy('fascia_anagrafica') \
                    .show()

+-----------------+
fascia_anagrafica|
+-----------------+
 16-19|
 20-29|
 30-39|
 40-49|
 50-59|
 60-69|
 70-79|
 80-89|
 90+|
+-----------------+

Si osserva che rispetto alla tabella popolazione_regioni manca la `fascia_anagrafica` 0-15

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_vaccini_somministrazioni.describe(['prima_dose', 'seconda_dose']) \
                             .show()

+-------+------------------+------------------+
summary| prima_dose| seconda_dose|
+-------+------------------+------------------+
 count| 42748| 42748|
 mean|328.72164779638814|137.94502666791428|
 stddev|1000.9215678340463| 558.9414535757198|
 min| 0| 0|
 max| 29498| 19243|
+-------+------------------+------------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', 'categoria_60_69', 'categoria_70_79']) \
                             .show()

+-------+------------------------------------------+---------------------------------+--------------------+-----------------+-----------------+
summary|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa| categoria_60_69| categoria_70_79|
+-------+------------------------------------------+---------------------------------+--------------------+-----------------+-----------------+
 count| 42748| 42748| 42748| 42748| 42748|
 mean| 75.81898568354075| 20.82904463366707| 15.316506035370075| 25.2064424066623|69.76822307476372|
 stddev| 226.9536664572629| 66.15868493516085| 61.102737050086844|325.9967518509442|679.4798418900963|
 min| 0| 0| 0| 0| 0|
 max| 4662| 2303| 2317| 20021| 28033|
+-------+------------------------------------------+---------------------------------+--------------------+-----------------+-----------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', 'categoria_soggetti_fragili', 'categoria_altro']) \
                             .show()

+-------+------------------+----------------------+------------------------------+--------------------------+-----------------+
summary| categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili| categoria_altro|
+-------+------------------+----------------------+------------------------------+--------------------------+-----------------+
 count| 42748| 42748| 42748| 42748| 42748|
 mean|144.24272480583886| 7.601548610461308| 27.258023767193787| 72.07352390755123|8.551651539253298|
 stddev| 911.4114589394183| 42.6170651430661| 151.6780634008098| 312.05108674408626|39.96799868320293|
 min| 0| 0| 0| 0| 0|
 max| 31064| 1214| 3583| 10910| 2008|
+-------+------------------+----------------------+------------------------------+--------------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_vaccini_somministrazioni.select([count(when(isnull(c), c)).alias(c) for c in dati_vaccini_somministrazioni.drop('data').columns]).show()

+---------------------+---------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
data_somministrazione|fornitore|fascia_anagrafica|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2|nome_area|
+---------------------+---------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+---------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+

Si verifica la presenza di *missing values* nella serie temporale.

In [0]:
min_data = dati_vaccini_somministrazioni.agg(min('data_somministrazione').alias('minimo'))
max_data = dati_vaccini_somministrazioni.agg(max('data_somministrazione').alias('massimo'))

date = min_data.join(max_data)

date.show()

+----------+----------+
 minimo| massimo|
+----------+----------+
2020-12-27|2021-04-30|
+----------+----------+

In [0]:
time_series = date.withColumn('data_somministrazione', explode(expr('sequence(minimo, massimo, interval 1 day)'))) \
                  .select('data_somministrazione')

time_series.show()

+---------------------+
data_somministrazione|
+---------------------+
 2020-12-27|
 2020-12-28|
 2020-12-29|
 2020-12-30|
 2020-12-31|
 2021-01-01|
 2021-01-02|
 2021-01-03|
 2021-01-04|
 2021-01-05|
 2021-01-06|
 2021-01-07|
 2021-01-08|
 2021-01-09|
 2021-01-10|
 2021-01-11|
 2021-01-12|
 2021-01-13|
 2021-01-14|
 2021-01-15|
+---------------------+
only showing top 20 rows

In [0]:
date_covid = dati_vaccini_somministrazioni.select('data_somministrazione') \
                               .withColumn('data_somministrazione', date_format(dati_vaccini_somministrazioni.data_somministrazione, 'yyyy-MM-dd').cast('date')) \
                               .distinct() \
                               .orderBy('data_somministrazione')

date_covid.join(time_series, on='data_somministrazione', how='anti') \
          .show()

+---------------------+
data_somministrazione|
+---------------------+
+---------------------+

## Elaborazione

### Dati covid Regioni

#### Attuali Positivi per Regione

In [0]:
attuali_positivi_regione = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data','denominazione_regione', 'totale_positivi') \
                                             .withColumnRenamed('denominazione_regione', 'regione') \
                                             .withColumnRenamed('totale_positivi', 'attuali_positivi')

popolazione = popolazione_regioni.groupBy('denominazione_regione') \
                                 .sum('totale_generale')

cond = [popolazione.denominazione_regione == attuali_positivi_regione.regione]
attuali_positivi_regione = attuali_positivi_regione.join(popolazione, cond, how='left') \
                                                   .drop('denominazione_regione') \
                                                   .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                   .orderBy('regione')

attuali_positivi_regione = attuali_positivi_regione.withColumn('positivi_per_100k', (attuali_positivi_regione.attuali_positivi / attuali_positivi_regione.popolazione * 100000).cast('int')) \
                                                   .orderBy('positivi_per_100k', ascending=False)

attuali_positivi_regione.show(21)

+----------+--------------------+----------------+-----------+-----------------+
 data| regione|attuali_positivi|popolazione|positivi_per_100k|
+----------+--------------------+----------------+-----------+-----------------+
2021-04-30| Campania| 91388| 5712143| 1599|
2021-04-30| Puglia| 47592| 3953305| 1203|
2021-04-30| Basilicata| 6230| 553254| 1126|
2021-04-30| Sardegna| 17093| 1611621| 1060|
2021-04-30| Emilia-Romagna| 44356| 4464119| 993|
2021-04-30| Calabria| 15214| 1894110| 803|
2021-04-30| Lazio| 44696| 5755700| 776|
2021-04-30| Abruzzo| 8383| 1293941| 647|
2021-04-30|Friuli Venezia Gi...| 7578| 1206216| 628|
2021-04-30| Valle d'Aosta| 749| 125034| 599|
2021-04-30| Toscana| 20669| 3692555| 559|
2021-04-30| Lombardia| 52485| 10027602| 523|
2021-04-30| Sicilia| 24896| 4875290| 510|
2021-04-30| Veneto| 21892| 4879133| 448|
2021-04-30| Marche| 6202| 1512672| 410|
2021-04-30| Piemonte| 15699| 4311217| 364|
2021-04-30| Liguria| 5471| 1524826| 358|
2021-04-30| Umbria| 2933| 870165| 337|
2021-04-30| Molise| 629| 300516| 209|
2021-04-30| Trento| 1076| 545425| 197|
2021-04-30| Bolzano| 1039| 532644| 195|
+----------+--------------------+----------------+-----------+-----------------+

#### Panoramica Casi per Mese

In [0]:
casi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'nuovi_positivi') \
                  .groupby('mese') \
                  .sum('nuovi_positivi') \
                  .withColumnRenamed('sum(nuovi_positivi)', 'casi')

In [0]:
tamponi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'tamponi') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('tamponi') \
                                    .groupby('mese') \
                                    .sum('max(tamponi)') \
                                    .withColumnRenamed('sum(max(tamponi))', 'tamponi')

my_window = Window.partitionBy().orderBy("mese") 

tamponi_mensili = tamponi_mensili.withColumn("prev_value", lag(tamponi_mensili.tamponi) \
                                 .over(my_window)) 
tamponi_mensili = tamponi_mensili.withColumn("diff", when(isnull(tamponi_mensili.tamponi - tamponi_mensili.prev_value), tamponi_mensili.tamponi).otherwise(tamponi_mensili.tamponi - tamponi_mensili.prev_value)) \
                                 .drop('tamponi', 'prev_value') \
                                 .withColumnRenamed('diff', 'tamponi')

In [0]:
panoramica_mensile = casi_mensili.join(tamponi_mensili, on='mese') \
                                 .orderBy('mese', ascending=False)

panoramica_mensile = panoramica_mensile.withColumn('percentuale_tamponi_positivi', panoramica_mensile.casi / panoramica_mensile.tamponi * 100)

panoramica_mensile.show()

# path = "dbfs:/FileStore/gu.dichiara@studenti.unina.it/panoramica_mensile.csv"
# panoramica_mensile.write.csv(path, mode='overwrite', sep=',', header=True)

+-------+------+-------+----------------------------+
 mese| casi|tamponi|percentuale_tamponi_positivi|
+-------+------+-------+----------------------------+
2021-04|439446|8677255| 5.064343504944825|
2021-03|648200|9769610| 6.634860552263601|
2021-02|372503|7466161| 4.989217350121435|
2021-01|445585|5948709| 7.4904487679595695|
2020-12|508914|4772918| 10.662533904835575|
2020-11|922206|6160638| 14.969326228874348|
2020-10|364607|4450539| 8.192423434554781|
2020-09| 45623|2689063| 1.6966132812805055|
2020-08| 21702|1831746| 1.1847712510358968|
2020-07| 7006|1423003| 0.4923390885331935|
2020-06| 7584|1511371| 0.5017960513996894|
2020-05| 27556|1899522| 1.4506807502097896|
2020-04| 99671|1472249| 6.769982523336745|
2020-03|104664| 488307| 21.434056853577772|
2020-02| 1120| 18661| 6.001821981673007|
+-------+------+-------+----------------------------+

#### Dati Critici per Mese

In [0]:
decessi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'deceduti') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('deceduti') \
                                    .groupby('mese') \
                                    .sum('max(deceduti)') \
                                    .withColumnRenamed('sum(max(deceduti))', 'deceduti')

my_window = Window.partitionBy().orderBy("mese") 

decessi_mensili = decessi_mensili.withColumn("prev_value", lag(decessi_mensili.deceduti) \
                                 .over(my_window)) 
decessi_mensili = decessi_mensili.withColumn("diff", when(isnull(decessi_mensili.deceduti - decessi_mensili.prev_value), decessi_mensili.deceduti).otherwise(decessi_mensili.deceduti - decessi_mensili.prev_value)) \
                                 .drop('deceduti', 'prev_value') \
                                 .withColumnRenamed('diff', 'deceduti')

In [0]:
terapie_intensive_mensili = dati_covid_regioni.select('data', 'terapia_intensiva') \
                  .groupby('data') \
                  .sum('terapia_intensiva') \
                  .select(date_format('data','yyyy-MM').alias('mese'), 'sum(terapia_intensiva)') \
                  .groupby('mese') \
                  .mean('sum(terapia_intensiva)') \
                  .withColumnRenamed('avg(sum(terapia_intensiva))', 'media_terapia_intensiva')

terapie_intensive_mensili = terapie_intensive_mensili.withColumn('media_terapia_intensiva', terapie_intensive_mensili.media_terapia_intensiva.cast('int'))

In [0]:
dati_critici_mensili = decessi_mensili.join(casi_mensili, on='mese') \
                                      .join(terapie_intensive_mensili, on='mese') \
                                      .orderBy('mese', ascending=False)

dati_critici_mensili.show()

+-------+--------+------+-----------------------+
 mese|deceduti| casi|media_terapia_intensiva|
+-------+--------+------+-----------------------+
2021-04| 11461|439446| 3306|
2021-03| 11647|648200| 3127|
2021-02| 9183|372503| 2128|
2021-01| 14357|445585| 2483|
2020-12| 18583|508914| 3005|
2020-11| 16958|922206| 3231|
2020-10| 2724|364607| 785|
2020-09| 411| 45623| 194|
2020-08| 341| 21702| 58|
2020-07| 311| 7006| 57|
2020-06| 1406| 7584| 207|
2020-05| 5458| 27556| 882|
2020-04| 15539| 99671| 2975|
2020-03| 12399|104664| 1985|
2020-02| 29| 1120| 53|
+-------+--------+------+-----------------------+

### Dati Somministrazioni Vaccini

#### Percentuale Popolazione Vaccinata per Regione
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione.

In [0]:
dosi_somministrate_regione = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('nome_area') \
                                                          .withColumnRenamed('nome_area', 'regione') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale')

cond = [popolazione.denominazione_regione == dosi_somministrate_regione.regione]
dosi_somministrate_regione = dosi_somministrate_regione.join(popolazione, cond, how='left') \
                                                       .drop('denominazione_regione') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .orderBy('regione')

dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_prima_dose', dosi_somministrate_regione.vaccinati_prima_dose / dosi_somministrate_regione.popolazione * 100)
dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_seconda_dose', dosi_somministrate_regione.vaccinati_seconda_dose / dosi_somministrate_regione.popolazione * 100)

dosi_somministrate_regione.show(21)

+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 315821| 126627| 1293941| 24.407681648545026| 9.786149445762984|
 Basilicata| 131193| 54155| 553254| 23.71297812577948| 9.788451597277199|
 Bolzano| 136324| 51672| 532644| 25.593830025307714| 9.701038592380652|
 Calabria| 379280| 161775| 1894110| 20.024180221845615| 8.540950631167146|
 Campania| 1237505| 477110| 5712143| 21.664461131312716| 8.352557000061097|
 Emilia-Romagna| 1110377| 532111| 4464119| 24.87337367126638| 11.919731530454273|
Friuli Venezia Gi...| 283762| 135626| 1206216| 23.52497396817817| 11.243923144776723|
 Lazio| 1324507| 590965| 5755700| 23.012092360616432| 10.267473982313186|
 Liguria| 423584| 190865| 1524826| 27.77916955770691| 12.517165893026483|
 Lombardia| 2419930| 955480| 10027602| 24.132688951954815| 9.528499435857148|
 Marche| 400176| 153045| 1512672| 26.454908929364727| 10.11752713079901|
 Molise| 75377| 40223| 300516| 25.08252472414114| 13.384645077133998|
 Piemonte| 1088512| 460401| 4311217| 25.248369543913007| 10.679142339622432|
 Puglia| 968258| 317182| 3953305| 24.49236777835254| 8.02321095893183|
 Sardegna| 354594| 152565| 1611621| 22.002319403879696| 9.466555722468248|
 Sicilia| 986820| 445884| 4875290| 20.24125744314697| 9.145794404025196|
 Toscana| 878416| 390248| 3692555| 23.78883997665573| 10.568508796754552|
 Trento| 135223| 42070| 545425| 24.792226245588303| 7.713251134436448|
 Umbria| 215876| 90102| 870165| 24.80862824866548| 10.354587922980125|
 Valle d'Aosta| 33188| 12352| 125034| 26.54318025497065| 9.878912935681495|
 Veneto| 1153470| 516416| 4879133| 23.640880459704626| 10.584175508230663|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per fascia anagrafica.

In [0]:
dosi_somministrate_fascia = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('range_eta').sum('totale_generale')

cond = [popolazione.range_eta == dosi_somministrate_fascia.fascia_anagrafica]
dosi_somministrate_fascia = dosi_somministrate_fascia.join(popolazione, cond, how='left') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .drop('range_eta') \
                                                       .orderBy('fascia_anagrafica')

dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_prima_dose', dosi_somministrate_fascia.vaccinati_prima_dose / dosi_somministrate_fascia.popolazione * 100)
dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_seconda_dose', dosi_somministrate_fascia.vaccinati_seconda_dose / dosi_somministrate_fascia.popolazione * 100)

dosi_somministrate_fascia.show(9)

+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 16-19| 28153| 9250| 2298846| 1.2246579370692947| 0.40237580072784346|
 20-29| 567939| 286377| 6084382| 9.334374468927164| 4.70675575596667|
 30-39| 820538| 382754| 6854632| 11.970562387594258| 5.583873795121313|
 40-49| 1203438| 494618| 8937229| 13.46544885444918| 5.534355223526219|
 50-59| 1678332| 679190| 9414195| 17.827674060288746| 7.214530822869082|
 60-69| 2287448| 570961| 7364364| 31.06103935112387| 7.7530252442709235|
 70-79| 3603044| 538469| 5968373| 60.368948120367136| 9.022040009898845|
 80-89| 3157181| 2413368| 3628160| 87.01879189451402| 66.51768389486682|
 90+| 706120| 521887| 791543| 89.20804049811571| 65.93286782903772|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Regione e Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione e fascia anagrafica.

In [0]:
dosi_somministrate = dati_vaccini_somministrazioni.groupBy('nome_area', 'fascia_anagrafica') \
                                                  .sum('prima_dose', 'seconda_dose') \
                                                  .orderBy('nome_area', 'fascia_anagrafica') \
                                                  .withColumnRenamed('nome_area', 'regione') \
                                                  .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                  .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

cond = [popolazione_regioni.denominazione_regione == dosi_somministrate.regione, popolazione_regioni.range_eta == dosi_somministrate.fascia_anagrafica]
dosi_somministrate = dosi_somministrate.join(popolazione_regioni.select('denominazione_regione', 'range_eta', 'totale_generale'), cond, how='left') \
                                       .drop('denominazione_regione', 'range_eta') \
                                       .withColumnRenamed('totale_generale', 'popolazione') \
                                       .orderBy('regione', 'fascia_anagrafica')

dosi_somministrate = dosi_somministrate.withColumn('percentuale_prima_dose', dosi_somministrate.vaccinati_prima_dose / dosi_somministrate.popolazione * 100)
dosi_somministrate = dosi_somministrate.withColumn('percentuale_seconda_dose', dosi_somministrate.vaccinati_seconda_dose / dosi_somministrate.popolazione * 100)

dosi_somministrate.show(189)

+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 16-19| 656| 241| 46751| 1.403178541635473| 0.5154969947166905|
 Abruzzo| 20-29| 11911| 5000| 130083| 9.15646164371978| 3.843699791671471|
 Abruzzo| 30-39| 18653| 8190| 150713| 12.376503685813434| 5.434169580593578|
 Abruzzo| 40-49| 28849| 9893| 190060| 15.178890876565296| 5.205198358413133|
 Abruzzo| 50-59| 42892| 12920| 204157| 21.009321257659547| 6.328462898651528|
 Abruzzo| 60-69| 61382| 12356| 167705| 36.60117468173281| 7.367699233773591|
 Abruzzo| 70-79| 67243| 9147| 130572| 51.49878993965016| 7.005330392427167|
 Abruzzo| 80-89| 68713| 56567| 84488| 81.32870940251871| 66.9527033424865|
 Abruzzo| 90+| 15522| 12313| 19515| 79.53881629515756| 63.09505508583141|
 Basilicata| 16-19| 485| 39| 22032| 2.2013435003631083| 0.1770152505446623|
 Basilicata| 20-29| 4744| 1281| 61183| 7.753787816877238| 2.0937188434695915|
 Basilicata| 30-39| 7858| 2959| 65204| 12.051407889086558| 4.538065149377339|
 Basilicata| 40-49| 13080| 4097| 78467| 16.669427912370804| 5.221303223010947|
 Basilicata| 50-59| 19046| 5155| 87822| 21.687048803261142| 5.869827605839084|
 Basilicata| 60-69| 22668| 4175| 72817| 31.130093247455953| 5.733551231168546|
 Basilicata| 70-79| 24590| 1210| 51805| 47.466460766335295| 2.335681883988032|
 Basilicata| 80-89| 31100| 28515| 36107| 86.13288282050571| 78.97360622593956|
 Basilicata| 90+| 7622| 6724| 7823| 97.43065320209638| 85.95168094081555|
 Bolzano| 16-19| 365| 135| 23269| 1.5686105977910525| 0.5801710430186084|
 Bolzano| 20-29| 6032| 2863| 61094| 9.87330998134023| 4.68622123285429|
 Bolzano| 30-39| 8462| 3574| 63127| 13.404723810730749| 5.661602800703344|
 Bolzano| 40-49| 12769| 5360| 75664| 16.87592514273631| 7.0839500951575385|
 Bolzano| 50-59| 17009| 6874| 83539| 20.360550162199694| 8.228492081542752|
 Bolzano| 60-69| 32525| 4110| 57041| 57.02038884311285| 7.205343524833014|
 Bolzano| 70-79| 31721| 5439| 46613| 68.05183103426083| 11.668418681483708|
 Bolzano| 80-89| 22332| 18818| 27108| 82.38158477202302| 69.41862180906006|
 Bolzano| 90+| 5109| 4499| 6165| 82.87104622871047| 72.9764801297648|
 Calabria| 16-19| 770| 280| 76743| 1.0033488396335821| 0.3648541235031208|
 Calabria| 20-29| 13100| 5977| 213889| 6.124672143027459| 2.79444010678436|
 Calabria| 30-39| 23001| 11642| 236668| 9.718677641252725| 4.919127216184697|
 Calabria| 40-49| 34309| 14609| 266417| 12.877931963801108| 5.483508935240619|
 Calabria| 50-59| 50751| 20639| 286804| 17.6953598973515| 7.196203679167655|
 Calabria| 60-69| 84969| 24183| 241215| 35.22542130464523| 10.025495926870219|
 Calabria| 70-79| 80250| 16735| 175208| 45.802703072919044| 9.551504497511528|
 Calabria| 80-89| 76232| 56329| 107720| 70.76865948756034| 52.29205347196435|
 Calabria| 90+| 15898| 11381| 23058| 68.94787058721485| 49.358140341746896|
 Campania| 16-19| 3551| 1198| 261694| 1.3569283208632983| 0.45778657516030175|
 Campania| 20-29| 53102| 24637| 693479| 7.65733353136865| 3.552667059853291|
 Campania| 30-39| 80894| 36263| 715258| 11.309765147680977| 5.069918826493377|
 Campania| 40-49| 117968| 43183| 835597| 14.117810379884082| 5.167921857067462|
 Campania| 50-59| 178825| 61567| 868684| 20.5857365854557| 7.087387358348951|
 Campania| 60-69| 276850| 69851| 670867| 41.26749415308847| 10.412048885993796|
 Campania| 70-79| 292292| 55668| 484380| 60.34353193773484| 11.492629753499319|
 Campania| 80-89| 198475| 160737| 255273| 77.75009499633725| 62.96670623215146|
 Campania| 90+| 35548| 24006| 49044| 72.48185302993231| 48.9478835331539|
 Emilia-Romagna| 16-19| 2404| 757| 160045| 1.5020775406916804| 0.47299197100815393|
 Emilia-Romagna| 20-29| 45581| 24516|

#### Categorie

In [0]:
dosi_somministrate_categorie_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), \
                                                                            'categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', \
                                                                            'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', \
                                                                            'categoria_soggetti_fragili', 'categoria_altro') \
                                                                    .groupby('mese') \
                                                                    .agg(sum('categoria_operatori_sanitari_sociosanitari').alias('operatori_sanitari_sociosanitari'), \
                                                                         sum('categoria_personale_non_sanitario').alias('personale_non_sanitario'), \
                                                                         sum('categoria_ospiti_rsa').alias('ospiti_rsa'), \
                                                                         sum('categoria_60_69').alias('60_69'), \
                                                                         sum('categoria_70_79').alias('70_79'), \
                                                                         sum('categoria_over80').alias('over80'), \
                                                                         sum('categoria_forze_armate').alias('forze_armate'), \
                                                                         sum('categoria_personale_scolastico').alias('personale_scolastico'), \
                                                                         sum('categoria_soggetti_fragili').alias('soggetti_fragili'), \
                                                                         sum('categoria_altro').alias('altro')) \
                                                                      .orderBy('mese')

dosi_somministrate_categorie_mensili.show()

+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa| 60_69| 70_79| over80|forze_armate|personale_scolastico|soggetti_fragili| altro|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
2020-12| 33953| 1493| 3262| 163| 42| 51| 0| 8| 1| 843|
2021-01| 1458515| 229726| 200902| 11933| 4473| 17783| 177| 324| 142| 55653|
2021-02| 901180| 269863| 212599| 14417| 8365| 639851| 67406| 176823| 2618| 76545|
2021-03| 598595| 280859| 154729| 86882| 401482|2823462| 215360| 847375| 439056|116588|
2021-04| 248867| 108459| 83258|964130|2568090|2684941| 42008| 140696| 2639182|115937|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+

In [0]:
dosi_somministrate_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), 'prima_dose', 'seconda_dose') \
                                                          .groupby('mese') \
                                                          .sum('prima_dose', 'seconda_dose')

dosi_somministrate_mensili = dosi_somministrate_mensili.withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                       .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')


totale_dosi_somministrate_mensili = dosi_somministrate_mensili.withColumn('totale_dosi', dosi_somministrate_mensili.vaccinati_prima_dose + dosi_somministrate_mensili.vaccinati_seconda_dose)

percentuale_dosi_categorie = totale_dosi_somministrate_mensili.join(dosi_somministrate_categorie_mensili, on='mese') \
                                                              .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                              .withColumn('operatori_sanitari_sociosanitari', round(dosi_somministrate_categorie_mensili.operatori_sanitari_sociosanitari / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_non_sanitario', round(dosi_somministrate_categorie_mensili.personale_non_sanitario / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('ospiti_rsa', round(dosi_somministrate_categorie_mensili.ospiti_rsa / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('60_69', round(dosi_somministrate_categorie_mensili['60_69'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('70_79', round(dosi_somministrate_categorie_mensili['70_79'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('over80', round(dosi_somministrate_categorie_mensili.over80 / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('forze_armate', round(dosi_somministrate_categorie_mensili.forze_armate / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_scolastico', round(dosi_somministrate_categorie_mensili.personale_scolastico / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('soggetti_fragili', round(dosi_somministrate_categorie_mensili.soggetti_fragili / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('altro', round(dosi_somministrate_categorie_mensili.altro / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .drop('totale_dosi') \
                                                              .orderBy('mese')
    
percentuale_dosi_categorie.show()

+-------+--------------------------------+-----------------------+----------+------+------+------+------------+--------------------+----------------+-----+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa| 60_69| 70_79|over80|forze_armate|personale_scolastico|soggetti_fragili|altro|
+-------+--------------------------------+-----------------------+----------+------+------+------+------------+--------------------+----------------+-----+
2020-12| 85.275| 3.75| 8.193| 0.409| 0.105| 0.128| 0.0| 0.02| 0.003|2.117|
2021-01| 73.676| 11.605| 10.148| 0.603| 0.226| 0.898| 0.009| 0.016| 0.007|2.811|
2021-02| 38.03| 11.388| 8.972| 0.608| 0.353|27.002| 2.845| 7.462| 0.11| 3.23|
2021-03| 10.036| 4.709| 2.594| 1.457| 6.731|47.339| 3.611| 14.207| 7.361|1.955|
2021-04| 2.594| 1.13| 0.868|10.048|26.763|27.981| 0.438| 1.466| 27.504|1.208|
+-------+--------------------------------+-----------------------+----------+------+------+------+------------+--------------------+----------------+-----+

#### Fornitori e Fasce Anagrafiche

In [0]:
dosi_fascia_fornitore = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica', 'fornitore') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica', 'fornitore') 

totale_dosi_somministrate_fascia = dosi_somministrate_fascia.select('fascia_anagrafica', 'vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                            .withColumn('totale_dosi', dosi_somministrate_fascia.vaccinati_prima_dose + dosi_somministrate_fascia.vaccinati_seconda_dose) \
                                                            .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose')

dosi_fascia_fornitore = dosi_fascia_fornitore.join(totale_dosi_somministrate_fascia, on='fascia_anagrafica')

dosi_fascia_fornitore = dosi_fascia_fornitore.withColumn('percentuale_dosi', round((dosi_fascia_fornitore['sum(prima_dose)'] + dosi_fascia_fornitore['sum(seconda_dose)']) *100 /  dosi_fascia_fornitore.totale_dosi, 3)) \
                                             .drop('sum(prima_dose)', 'sum(seconda_dose)', 'totale_dosi') \
                                             .orderBy('fascia_anagrafica', 'fornitore')

dosi_fascia_fornitore.show(100)

+-----------------+--------------------+----------------+
fascia_anagrafica| fornitore|percentuale_dosi|
+-----------------+--------------------+----------------+
 16-19| Janssen| 0.032|
 16-19| Moderna| 7.398|
 16-19| Pfizer/BioNTech| 85.806|
 16-19|Vaxzevria (AstraZ...| 6.764|
 20-29| Janssen| 0.176|
 20-29| Moderna| 4.492|
 20-29| Pfizer/BioNTech| 74.67|
 20-29|Vaxzevria (AstraZ...| 20.662|
 30-39| Janssen| 0.16|
 30-39| Moderna| 4.464|
 30-39| Pfizer/BioNTech| 70.325|
 30-39|Vaxzevria (AstraZ...| 25.052|
 40-49| Janssen| 0.118|
 40-49| Moderna| 4.81|
 40-49| Pfizer/BioNTech| 67.071|
 40-49|Vaxzevria (AstraZ...| 28.001|
 50-59| Janssen| 0.086|
 50-59| Moderna| 5.273|
 50-59| Pfizer/BioNTech| 69.584|
 50-59|Vaxzevria (AstraZ...| 25.057|
 60-69| Janssen| 0.711|
 60-69| Moderna| 6.375|
 60-69| Pfizer/BioNTech| 61.828|
 60-69|Vaxzevria (AstraZ...| 31.086|
 70-79| Janssen| 0.734|
 70-79| Moderna| 7.18|
 70-79| Pfizer/BioNTech| 48.78|
 70-79|Vaxzevria (AstraZ...| 43.306|
 80-89| Janssen| 0.01|
 80-89| Moderna| 9.496|
 80-89| Pfizer/BioNTech| 89.352|
 80-89|Vaxzevria (AstraZ...| 1.142|
 90+| Janssen| 0.007|
 90+| Moderna| 13.803|
 90+| Pfizer/BioNTech| 85.511|
 90+|Vaxzevria (AstraZ...| 0.678|
+-----------------+--------------------+----------------+

### Correlazione Dati

#### Dosi Somministrate e Media terapie intensive

In [0]:
dosi_somministrate_terapie_intensive = dosi_somministrate_mensili.join(terapie_intensive_mensili, on='mese', how='left') \
                                                                 .orderBy('mese', ascending=False)


dosi_somministrate_terapie_intensive.show()

+-------+--------------------+----------------------+-----------------------+
 mese|vaccinati_prima_dose|vaccinati_seconda_dose|media_terapia_intensiva|
+-------+--------------------+----------------------+-----------------------+
2021-04| 6935689| 2659879| 3306|
2021-03| 4149031| 1815357| 3127|
2021-02| 1588351| 781316| 2128|
2021-01| 1339306| 640322| 2483|
2020-12| 39816| 0| 3005|
+-------+--------------------+----------------------+-----------------------+

#### Dosi Somministrate e Media terapie intensive

In [0]:
attuali_terapie_intensive_regioni = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data', 'denominazione_regione', 'terapia_intensiva') \
                                             .withColumnRenamed('denominazione_regione', 'regione')

In [0]:
attuali_dosi_somministrate_terapie_intensive = attuali_terapie_intensive_regioni.join(dosi_somministrate_regione, on='regione') \
                                                                                .select('data', 'regione', 'percentuale_prima_dose', 'percentuale_seconda_dose', 'terapia_intensiva') \
                                                                                .orderBy('regione')

attuali_dosi_somministrate_terapie_intensive.show(21)

+----------+--------------------+----------------------+------------------------+-----------------+
 data| regione|percentuale_prima_dose|percentuale_seconda_dose|terapia_intensiva|
+----------+--------------------+----------------------+------------------------+-----------------+
2021-04-30| Abruzzo| 24.407681648545026| 9.786149445762984| 36|
2021-04-30| Basilicata| 23.71297812577948| 9.788451597277199| 8|
2021-04-30| Bolzano| 25.593830025307714| 9.701038592380652| 2|
2021-04-30| Calabria| 20.024180221845615| 8.540950631167146| 46|
2021-04-30| Campania| 21.664461131312716| 8.352557000061097| 137|
2021-04-30| Emilia-Romagna| 24.87337367126638| 11.919731530454273| 226|
2021-04-30|Friuli Venezia Gi...| 23.52497396817817| 11.243923144776723| 30|
2021-04-30| Lazio| 23.012092360616432| 10.267473982313186| 284|
2021-04-30| Liguria| 27.77916955770691| 12.517165893026483| 65|
2021-04-30| Lombardia| 24.132688951954815| 9.528499435857148| 550|
2021-04-30| Marche| 26.454908929364727| 10.11752713079901| 67|
2021-04-30| Molise| 25.08252472414114| 13.384645077133998| 8|
2021-04-30| Piemonte| 25.248369543913007| 10.679142339622432| 210|
2021-04-30| Puglia| 24.49236777835254| 8.02321095893183| 224|
2021-04-30| Sardegna| 22.002319403879696| 9.466555722468248| 49|
2021-04-30| Sicilia| 20.24125744314697| 9.145794404025196| 165|
2021-04-30| Toscana| 23.78883997665573| 10.568508796754552| 253|
2021-04-30| Trento| 24.792226245588303| 7.713251134436448| 20|
2021-04-30| Umbria| 24.80862824866548| 10.354587922980125| 34|
2021-04-30| Valle d'Aosta| 26.54318025497065| 9.878912935681495| 8|
2021-04-30| Veneto| 23.640880459704626| 10.584175508230663| 161|
+----------+--------------------+----------------------+------------------------+-----------------+

# IDEE
* Dubito che metteremo mano, ma ci sono somministrazioni di vaccini a singola dose.